## Loading Data and Creating Vector Store

In [10]:
## RUN

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.llms import OpenAI
import chromadb
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from IPython.display import Markdown
# from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
import tqdm as notebook_tqdm
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [6]:
import sys
!{sys.executable} -m pip install llama-index-vector-stores-chroma
!{sys.executable} -m pip install llama-index-embeddings-huggingface  
!{sys.executable} -m pip install llama-index-llms-ollama
!{sys.executable} -m pip install chromadb sentence-transformers

SyntaxError: invalid syntax (3705398690.py, line 1)

In [3]:
import torch
torch.cuda.is_available()

False

In [11]:
## RUN

## Define Embedding Function
# Many ways of getting embeddings: https://docs.llamaindex.ai/en/latest/examples/embeddings/ollama_embedding/
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", device="cuda:0")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

## Define LLM
Settings.llm = Ollama(model="llama3", request_timeout=120.0)

## Define Splitter
Settings.node_parser = SentenceSplitter(chunk_size=2048, chunk_overlap=512)

## Define Other Settings
# number of tokens reserved for text generation.
Settings.num_output = 2048
# maximum input size to the LLM
Settings.context_window = 8000

Load the documents

In [5]:
## Load the PDFs
# https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

documents = SimpleDirectoryReader(
    input_dir="C:/Users/Gaurav Jetley/OneDrive - Colostate/RAG_LLM/Tests/health_it_om_papers",
    recursive=True
).load_data(num_workers=4)

ValueError: Directory C:/Users/Gaurav Jetley/OneDrive - Colostate/RAG_LLM/Tests/health_it_om_papers does not exist.

Use Chroma to store the embeddings from a VectorStoreIndex:

- initialize the Chroma client
- create a Collection to store your data in Chroma
- assign Chroma as the vector_store in a StorageContext
- initialize your VectorStoreIndex using that StorageContext


In [6]:
# Create Client and a new Collection (Vector Store)
# https://docs.llamaindex.ai/en/stable/understanding/storing/storing/

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db") # Persistant

# Create collection
chroma_collection = db.get_or_create_collection("DB_collection")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Change default storage context to new vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Creating a vector store index
index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context)
# Save the vector store index
index.storage_context.persist(persist_dir="./chroma_db/index_save")


NameError: name 'documents' is not defined

If you've already created and stored your embeddings, you'll want to load them directly without loading your documents or creating a new VectorStoreIndex:

- https://stackoverflow.com/questions/76321460/llamaindex-cannot-persist-index-to-chroma-db-and-load-later
- https://docs.llamaindex.ai/en/stable/understanding/storing/storing/

In [12]:
## RUN

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db") # Persistant

# Create collection
chroma_collection = db.get_or_create_collection("DB_collection")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Change default storage context to new vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir="./chroma_db/index_save") # <-- Give the persit dir used to save the index

# Load the vector store index from stored vectors
index = VectorStoreIndex.from_documents([], # <-- Change this to [] instead of documents if loading an already created and stored index
                                        storage_context=storage_context)


Create Query Engine

- https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/usage_pattern/
- https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/

In [8]:
## RUN

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    # response_mode="compact",
    # response_mode="refine",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
# Inserting New Documents or Nodes
# https://docs.llamaindex.ai/en/stable/module_guides/indexing/document_management/

# index = VectorStoreIndex([])
# for doc in documents:
#     index.insert(doc)

In [ ]:
# Metadata
# https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents/

## Regular RAG

In [13]:
## RUN

response = query_engine.query("Get me studies that have looked at knowledge management in hospitals and/or healthcare")
display(Markdown(f"{response}"))

ReadTimeout: timed out

In [27]:
response = query_engine.query("Create a literature review for health IT and healthcare technology on length of stay and readmissions")
display(Markdown(f"{response}"))

**Rewrite**

Hospital readmissions are a significant challenge in healthcare, with approximately one-third of patients being readmitted within 30 days of discharge. Length of stay (LOS) has been identified as an important factor in hospital readmissions, with longer LOS associated with increased risk of readmission.

The study examined trends in outpatient progress notes over a 10-year period at an academic medical center, finding that note bloat and redundancy were significant issues. The majority of note text was entered using templates and copy-paste rather than direct typing, which may contribute to the observed increases in LOS.

To address this issue, health information technology (HIT) and healthcare technology have the potential to improve patient outcomes by reducing LOS and readmissions. For instance, electronic health records (EHRs) can facilitate better communication among healthcare providers, leading to improved care coordination and reduced readmissions.

Several studies have investigated the relationship between LOS and readmissions in various patient populations. For example, a study published in JAMA found that longer LOS was associated with increased risk of readmission among patients hospitalized with acute myocardial infarction, heart failure, or pneumonia.

Furthermore, HIT and healthcare technology have been shown to reduce LOS and readmissions. For instance, a study published in the Journal of Hospital Medicine found that implementation of an EHR system reduced LOS by 15% and readmissions by 20%.

In conclusion, hospital readmissions are a significant challenge in healthcare, with length of stay playing an important role. HIT and healthcare technology have been shown to improve patient outcomes by reducing LOS and readmissions.

In [16]:
response = query_engine.query("What variables have been found to impact service time (LOS) in hospitals")
display(Markdown(f"{response}"))

According to the context, the variables that have been found to impact service time (LOS) in hospitals include:

* Urgent response strategies, such as attending physician workload
* PDO completion rate and crisis activation (CY)
* Occupancy rates
* Patient severity scores
* Insurance types
* Day of week FE
* Quarter FE
* Year FE
* Attending physician FE

In [10]:
response = query_engine.query("what types of instrumental varibales have been used in healthcare resaerch and which variables were instrumented (were endogeneous)")
display(Markdown(f"{response}"))

The two instrumental variables used in this research are the average LOS of nonfocal hospitalizations treated by the same physician in the same hospital during the same quarter, and the proportion of nonfocal condition hospitalizations with LOS > GMLOS, handled by the same physician. These instruments aim to capture the tendency of the physician to discharge early or late.

The endogenous variable that was instrumented is log(LOS/GMLOS), which is used as an explanatory variable in the analysis.

In [19]:
response = query_engine.query("Summarize the paper: Gatekeeping, Fast and Slow - An Empirical Study of Referral Errors in the Emergency Department")
display(Markdown(f"{response}"))

**Rewrite**

This paper explores the role of gatekeeping in emergency departments and its impact on system performance and patient outcomes. The study examines referral errors, congestion, and patient outcomes in EDs, shedding light on the trade-offs involved in designing a multistage gatekeeping process. The findings suggest that this approach can outperform traditional single-stage processes under specific conditions.

In [28]:
response = query_engine.query("What types of theories have been used ito explain the impact of certain variables on LOS or readmission rate")
display(Markdown(f"{response}"))

Based on the provided excerpt from the document "MS_Past the Point of Speeding Up - The Negative Effects of Workload Saturation.pdf", I can answer the query.

The theory that has been used to explain the impact of workload saturation, hospital-acquired conditions (HACs), and length of stay (LOS) on readmission rate is the causal mediation analysis framework. This framework is based on the concept of mediated effects, where an independent variable (in this case, HACs) affects a dependent variable (readmission rate) through one or more intermediate variables (LOS).

The theory suggests that HACs increase LOS, which in turn decreases the risk of readmission. The causal mediation analysis framework is used to estimate the total effect of HACs on readmission rate, as well as the mediated and direct effects of HACs on LOS.

Additionally, the document also mentions potential confounding variables (omitted variables) that could affect the estimated causal effects. To address this issue, sensitivity analyses are performed using the E-value approach, which estimates the minimum strength of association required for an unmeasured confounder to fully explain away a specific treatment-outcome association.

In [8]:
response = query_engine.query("What papers describe the impact of readmission rates. Create a comprehensive list.")
display(Markdown(f"{response}"))

Based on the provided context and candidate titles, I would suggest a comprehensive title that captures the essence of the analysis:

"Patient-, Visit-, and Physician-Level Characteristics and Comorbidities: A Profile of Factors Influencing Healthcare Utilization"

This title encompasses the various entities and themes present in the document, including patient-level variables, visit-level variables, comorbidities, geographic factors, and physician-level variables. It provides a comprehensive overview of the analysis, highlighting the relationships between these factors and healthcare utilization outcomes.

Here is the revised title:

"Patient-, Visit-, and Physician-Level Characteristics and Comorbidities: A Profile of Factors Influencing Healthcare Utilization"

As for papers describing the impact of readmission rates, here is a comprehensive list:

1. McNair PD, Luft HS (2012) Enhancing Medicare's hospital-acquired conditions policy to encompass readmissions. Medicare Medicaid Res Rev 2: mmrr.002.02.a03.
2. McDermott C, Stock GN (2007) Hospital operations and length of stay performance. Internat J Oper Prod Management 27(9):1020–1042.
3. McNair PD, Luft HS, Bindman AB (2009) Medicare's policy not to pay for treating hospital-acquired conditions: The impact. Health Affairs 28:1458–1493.
4. MedPAC (2004) Medicare payment advisory commission: Defining long-term care hospitals. Report to the Congress: New Approaches in Medicare (MedPAC, Washington, DC), 121–135.
5. Miller JL, Craighead CW, Karwan KR (2000) Service recovery: A framework and empirical investigation. J Oper Management 18(4):387–400.

Please note that these papers are not the only ones describing the impact of readmission rates but are a selection based on the provided context.

In [9]:
response = query_engine.query("List all instrumental variables (IV) used in healthcare resaerch")
display(Markdown(f"{response}"))

Based on the provided context, the following are the instrumental variables (IVs) used in healthcare research:

1. Preadmission service busyness
2. Service-TO-hospital busyness ratio
3. Average LOS of nonfocal hospitalizations treated by the same physician in the same hospital during the same quarter.
4. Proportion of nonfocal condition hospitalizations (i.e., conditions other than AMI, HF, and PN) with LOS > GMLOS, handled by the same physician.
5. Differential Distance (DD) variable
6. Binary variables based on high- or low-volume hospitals
7. HRR-level MU saturation

Note that some of these IVs may be used to correct for selection bias, control for unobservable factors, or address potential biases in correlations between certain variables and patient outcomes.

In [35]:
response = query_engine.query("Describe the effects of Reviewer Identity Disclosure on Product Sales")
display(Markdown(f"{response}"))

The provided context does not mention "Reviewer Identity Disclosure" or any related topic. The text discusses the direct and spillover effects of quality disclosure regulation in healthcare settings, particularly focusing on California hospitals. Therefore, I will not provide an answer based on this query as it is unrelated to the given context.

To answer a query accurately using the provided context information, I would need more relevant details about the topic or concept being discussed. If you could provide more context or clarify the query, I'll be happy to help!

## Agentic RAG

In [47]:
from llama_index.core import VectorStoreIndex, SummaryIndex

## HyDE (Hypothetical Document Embeddings)

https://docs.llamaindex.ai/en/stable/optimizing/advanced_retrieval/query_transformations/#hyde-hypothetical-document-embeddings

In [10]:
from llama_index.core.indices.query.query_transform.base import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

In [11]:
# run query with HyDE query transform

hyde = HyDEQueryTransform(include_original=True)

query_engine_hyde = TransformQueryEngine(query_engine, query_transform=hyde)


In [12]:
response = query_engine.query("Summarize the paper: Gatekeeping, Fast and Slow - An Empirical Study of Referral Errors in the Emergency Department")
display(Markdown(f"{response}"))

The study aims to investigate referral errors in emergency departments (EDs) and propose strategies for reducing such errors. The authors conducted an empirical analysis analyzing data from various sources, including journals like JAMA Internal Medicine, Journal of General Internal Medicine, and Emergency Medicine Journal.

The study focuses on three main themes: improving hospital efficiency, emergency department operations, and error reduction. It discusses the importance of optimizing emergency department operations to reduce errors and improve patient care. The authors review various studies that investigate strategies for improving hospital efficiency, patient flow, and error reduction in EDs.

Some key findings and insights from the study include:

* Gatekeeping processes play a crucial role in reducing referral errors.
* Alternative strategies, such as observation units and short-stay units, can be effective in managing acute medical conditions.
* Capacity planning is essential for optimizing emergency department operations and reducing errors.
* Error reduction methods, such as cognitive biases detection, can help improve decision-making in EDs.

The study also proposes a two-stage gatekeeping approach as a potential solution to manage overwhelmed settings. The authors provide an empirical analysis of referral errors in EDs, highlighting the importance of accurate referrals in patient care.

Overall, the study aims to provide a comprehensive review of existing literature on gatekeeping, fast and slow strategies, and error reduction methods in emergency departments.

In [13]:

response = query_engine_hyde.query("What papers describe the impact of readmission rates. Create a comprehensive list.")
display(Markdown(f"{response}"))

Based on the provided context, I've identified several papers that discuss the impact of readmission rates:

1. Krumholz HM, Normand S-LT, Keenan PS, Desai MM, Lin Z, Drye EE, Curtis JP, Bhat KR, Schreiner GC (2020) Hospital 30-day acute myocardial infarction readmission measure. Prepared for Centers for Medicare & Medicaid Services, Baltimore.
2. Joynt KE, Jha AK (2012) Thirty- day readmissions—truth and consequences. New England J. Medicine 366(15):1366–1369.
3. Joynt KE, Orav EJ, Jha AK (2011) Thirty-day readmission rates for Medicare beneficiaries by race and site of care. J. Amer. Med. Assoc. 305(7):675–681.
4. Kansagara DEH, Englander H, Salanitro A, Kagen D, Theobald C, Freeman M, Kripalani S (2011) Risk prediction models for hospital readmission: A systematic review. J. Amer. Medical Assoc. 306(15):1688–1698.
5. Bardhan et al. (2015) Predictive Analytics for Readmission of Patients with CHF

Additionally, other papers mentioned in the excerpt may also discuss the impact of readmission rates, but these five studies are specifically highlighted as describing the impact of readmission rates.

Please note that this list is based on the provided context and may not be an exhaustive or comprehensive list of all papers that describe the impact of readmission rates.

In [14]:
response = query_engine_hyde.query("What variables have been found to impact service time (LOS) in hospitals")
display(Markdown(f"{response}"))

According to the provided excerpt, variables that have been found to impact service time (Length of Stay or LOS) in hospitals include:

1. Attending Physicians' number of PDO Use: This variable has a positive and significant impact on LOS.
2. Crisis Activation (CY): This variable has a negative and statistically significant impact on LOS.
3. Hospitalist caseload
4. Process-of-care measures
5. Day-of-week effect
6. Admission day-of-the-week
7. Predicted Saturday discharge
8. Workload (as measured by census)
9. Cumulative fatigue or workload burden on service workers

## Multi-Step Query Transformations
https://docs.llamaindex.ai/en/stable/optimizing/advanced_retrieval/query_transformations/#multi-step-query-transformations
https://docs.llamaindex.ai/en/stable/examples/query_transformations/SimpleIndexDemo-multistep/

In [15]:
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine import MultiStepQueryEngine

In [16]:
step_decompose_transform = StepDecomposeQueryTransform(verbose=True)
query_engine_multistep = MultiStepQueryEngine(query_engine, query_transform=step_decompose_transform)

In [17]:
response = query_engine.query("Summarize the paper: Gatekeeping, Fast and Slow - An Empirical Study of Referral Errors in the Emergency Department")
display(Markdown(f"{response}"))

The paper "Gatekeeping, Fast and Slow - An Empirical Study of Referral Errors in the Emergency Department" investigates gatekeeping strategies for optimizing patient flow management in emergency departments. The study examines the impact of increasing ED congestion levels on gatekeeping errors, unnecessary hospitalization rates, and wrongful discharge rates in fast-only EDs versus fast-and-slow EDs. It also compares single-stage and two-stage gatekeeping systems, concluding that a two-stage system can be more effective in reducing errors and optimizing resource allocation.

The authors identify several key findings, including the common occurrence of referral errors in emergency departments, the effectiveness of a two- 

(Note: Please provide further instructions or clarification if needed)

In [50]:
response = query_engine_multistep.query("What papers describe the impact of readmission rates.")
display(Markdown(f"{response}"))

> Current query: What papers describe the impact of readmission rates.
> New query: None

The original question is asking about papers that describe the impact of readmission rates, but there is no context information provided to answer this question. Therefore, it is not possible to extract more information from the given context, and a new question cannot be created. The answer is "None".


Empty Response

In [68]:
response = query_engine_multistep.query("What variables have been found to impact service time (LOS) in hospitals")
display(Markdown(f"{response}"))

> Current query: What variables have been found to impact service time (LOS) in hospitals
> New query: I can extract information from this knowledge source!

Here's a subcomponent of the original question that I think can be answered:

What variables have been found to impact length of stay (LOS) in hospitals?

Answer: This question is relevant, and we can answer it based on the context. Since there is no specific information provided about LOS or hospitals, we can focus on general factors that might impact LOS. From a healthcare perspective, some common variables that have been found to impact LOS include:

* Patient characteristics (e.g., age, sex, comorbidities)
* Diagnosis and treatment complexity
* Hospital resources (e.g., staffing levels, bed availability)
* Clinical pathways and protocols
* Patient and family factors (e.g., patient satisfaction, caregiver involvement)

Please let me know if you'd like to explore more or refine this answer!
> Current query: What variables have b

This question is relevant, and we can answer it based on the context. Since there is no specific information provided about LOS or hospitals, we can focus on general factors that might impact LOS. From a healthcare perspective, some common variables that have been found to impact LOS include:

* Patient characteristics (e.g., age, sex, comorbidities)
* Diagnosis and treatment complexity
* Hospital resources (e.g., staffing levels, bed availability)

Please let me know if you'd like to explore more or refine this answer!

## Sub Question Query Engine
https://docs.llamaindex.ai/en/stable/examples/query_engine/sub_question_query_engine/

In [18]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [19]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
    ),
]

query_engine_subquestion = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

TypeError: __init__() missing 1 required positional argument: 'metadata'

## Metadata Extraction
https://docs.llamaindex.ai/en/stable/module_guides/indexing/metadata_extraction/
https://docs.llamaindex.ai/en/stable/examples/vector_stores/chroma_metadata_filter/
https://docs.llamaindex.ai/en/latest/examples/metadata_extraction/MarvinMetadataExtractorDemo/

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.llms import OpenAI
import chromadb
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from IPython.display import Markdown
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
import tqdm as notebook_tqdm

c:\anaconda\anaconda3\envs\Python38RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# We'll use GPU for the embedding model by using the device="cuda:0" option in huggingface.
# Ollama on GPU is a bit more involved: https://www.reddit.com/r/NixOS/comments/1atxzpe/how_to_enable_gpu_support_in_ollama/ 
import torch
torch.cuda.is_available()

True

In [7]:
## Define Embedding Function
# Many ways of getting embeddings: https://docs.llamaindex.ai/en/latest/examples/embeddings/ollama_embedding/
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", device="cuda:0")
# Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

## Define LLM
Settings.llm = Ollama(model="llama3", request_timeout=120.0)

## Define Splitter
Settings.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=128)

## Define Other Settings
Settings.num_output = 1024
Settings.context_window = 3900

In [4]:
from llama_index.core.extractors import (
    # SummaryExtractor,
    # QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

In [9]:
# When running the pipeline, we get an error. We run this code to avoid the error.
# https://github.com/run-llama/llama_index/issues/9978#:~:text=on%20Jan%2011-,I%27m%20not%20sure%20what%20the%20bot%20was%20going%20on%20about%2C%20but,import%20nest_asyncio%20%0Anest_asyncio.apply(),-%F0%9F%91%8D

# Note: For FastAPI, the code changes a bit
# https://github.com/run-llama/llama_index/issues/9978#:~:text=3%20weeks%20ago-,%40rohan%2Dpatankar%20in%20fastapi%2C%20you%20have%20to%20change%20the%20loop%20type,uvicorn.run(...%2C%20loop%3D%22asyncio%22),-peterdays%20commented%202

import nest_asyncio 
nest_asyncio.apply()

In [10]:
documents = SimpleDirectoryReader(
    input_dir="C:/Users/Gaurav Jetley/OneDrive - Colostate/RAG_LLM/Tests/health_it_om_papers",
    recursive=True
).load_data(num_workers=4)

In [11]:
# https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_metadata_extractor/#metadata-extraction-usage-pattern

transformations = [
    SentenceSplitter(chunk_size=1024, chunk_overlap=128),
    TitleExtractor(nodes=5), # extracts a title over the context of each Node
    # QuestionsAnsweredExtractor(questions=3), # extracts a set of questions that each Node can answer
    # SummaryExtractor(summaries=["prev", "self"]), # automatically extracts a summary over a set of Nodes
    KeywordExtractor(keywords=10), 
    EntityExtractor(prediction_threshold=0.5), # extracts entities (i.e. names of places, people, things) mentioned in the content of each Node
]

c:\anaconda\anaconda3\envs\Python38RAG\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
pipeline = IngestionPipeline(transformations=transformations)

nodes = pipeline.run(documents=documents)

Extracting entities: 100%|██████████| 1647/1647 [1:54:26<00:00,  4.17s/it]


In [28]:
# Examine the outputs
import random
samples = random.sample(nodes, 5)
for node in samples:
    print(node.metadata)

print(len(nodes))

print(nodes[1])

print(nodes[1].metadata)

{'page_label': '10', 'file_name': 'POMS_Missed Opportunities in Preventing Hospital Readmissions - Redesigning Post‐Discharge Checkup Policies.pdf', 'file_path': 'C:\\Users\\Gaurav Jetley\\OneDrive - Colostate\\RAG_LLM\\Tests\\health_it_om_papers\\POMS_Missed Opportunities in Preventing Hospital Readmissions - Redesigning Post‐Discharge Checkup Policies.pdf', 'file_type': 'application/pdf', 'file_size': 780707, 'creation_date': '2024-05-13', 'last_modified_date': '2023-02-02', 'document_title': 'Based on the provided information, I would suggest a comprehensive title that captures the essence of the document:\n\n**"Optimal Checkup Scheduling Strategies for Maximizing Objective Value: A Mathematical Approach to Improving Post-Discharge Healthcare Outcomes"**\n\nThis title incorporates key concepts and themes from the context, including:\n\n1. Optimal checkup scheduling\n2. Maximizing an objective value (improving healthcare outcomes)\n3. A mathematical approach\n\nThe title also highlig

In [31]:
# Finding problematic node with metadata not in the right format. Otherwise VectorStoreIndex gives an error: ValueError: Value for metadata entities must be one of (str, int, float, None)

# Assuming nodes is a list of TextNode objects
for i, node in enumerate(nodes):
    try:
        # Attempt to create an index with the single node
        index = VectorStoreIndex([node], storage_context=storage_context)
        print(f"Node {i} indexed successfully.")
    except ValueError as e:
        print(f"Error indexing node {i}: {e}")
        print("Problematic node metadata:", node.metadata)

Add of existing embedding ID: 0bfae0f3-3dc3-4569-8936-060a63eef066
Insert of existing embedding ID: 0bfae0f3-3dc3-4569-8936-060a63eef066


Node 0 indexed successfully.
Node 1 indexed successfully.
Node 2 indexed successfully.
Node 3 indexed successfully.
Node 4 indexed successfully.
Node 5 indexed successfully.
Node 6 indexed successfully.
Node 7 indexed successfully.
Node 8 indexed successfully.
Node 9 indexed successfully.
Node 10 indexed successfully.
Node 11 indexed successfully.
Node 12 indexed successfully.
Node 13 indexed successfully.
Node 14 indexed successfully.
Node 15 indexed successfully.
Error indexing node 16: Value for metadata entities must be one of (str, int, float, None)
Problematic node metadata: {'page_label': '76', 'file_name': '73.full.pdf', 'file_path': 'C:\\Users\\Gaurav Jetley\\OneDrive - Colostate\\RAG_LLM\\Tests\\health_it_om_papers\\73.full.pdf', 'file_type': 'application/pdf', 'file_size': 592760, 'creation_date': '2024-05-13', 'last_modified_date': '2023-10-01', 'document_title': 'Based on the provided text and potential title candidates, I suggest the following comprehensive title:\n\n**"A

In [36]:
nodes[16]

TextNode(id_='b811fdf0-38e5-4f64-8c9a-d4a669ef424e', embedding=None, metadata={'page_label': '76', 'file_name': '73.full.pdf', 'file_path': 'C:\\Users\\Gaurav Jetley\\OneDrive - Colostate\\RAG_LLM\\Tests\\health_it_om_papers\\73.full.pdf', 'file_type': 'application/pdf', 'file_size': 592760, 'creation_date': '2024-05-13', 'last_modified_date': '2023-10-01', 'document_title': 'Based on the provided text and potential title candidates, I suggest the following comprehensive title:\n\n**"Advancing Pulse Oximetry: Novel Device for Non-Invasive Hemoglobin and Oxygen Measurement"**\n\nThis title effectively captures the main themes and unique aspects of the study, including the development of a new device, its non-invasive measurement capabilities, and the advancement of pulse oximetry technology.', 'questions_this_excerpt_can_answer': 'Based on the provided excerpt, here are three potential questions that this context can provide specific answers to:\n\n1. What is the novel device developed 

In [34]:
# Sanitize one problematic node Metadata

def sanitize_metadata(metadata):
    sanitized_metadata = {}
    for key, value in metadata.items():
        if isinstance(value, (str, int, float, type(None))):
            sanitized_metadata[key] = value
        elif isinstance(value, list) or isinstance(value, dict):
            # Convert lists and dicts to string
            sanitized_metadata[key] = str(value)
        else:
            # Convert other types to string
            sanitized_metadata[key] = str(value)
    return sanitized_metadata

# Apply the sanitization function to the problematic node
problematic_node = nodes[16]
problematic_node.metadata = sanitize_metadata(problematic_node.metadata)

# Attempt to create the index again with the sanitized metadata
try:
    index = VectorStoreIndex([problematic_node], storage_context=storage_context)
    print("Node indexed successfully after sanitizing metadata.")
except ValueError as e:
    print(f"Error indexing node after sanitization: {e}")

Node indexed successfully after sanitizing metadata.


In [37]:
problematic_node

TextNode(id_='b811fdf0-38e5-4f64-8c9a-d4a669ef424e', embedding=None, metadata={'page_label': '76', 'file_name': '73.full.pdf', 'file_path': 'C:\\Users\\Gaurav Jetley\\OneDrive - Colostate\\RAG_LLM\\Tests\\health_it_om_papers\\73.full.pdf', 'file_type': 'application/pdf', 'file_size': 592760, 'creation_date': '2024-05-13', 'last_modified_date': '2023-10-01', 'document_title': 'Based on the provided text and potential title candidates, I suggest the following comprehensive title:\n\n**"Advancing Pulse Oximetry: Novel Device for Non-Invasive Hemoglobin and Oxygen Measurement"**\n\nThis title effectively captures the main themes and unique aspects of the study, including the development of a new device, its non-invasive measurement capabilities, and the advancement of pulse oximetry technology.', 'questions_this_excerpt_can_answer': 'Based on the provided excerpt, here are three potential questions that this context can provide specific answers to:\n\n1. What is the novel device developed 

In [35]:
# Sanitizing all node Metadata

for i, node in enumerate(nodes):
    node.metadata = sanitize_metadata(node.metadata)

In [4]:
# Create Client and a new Collection (Vector Store)
# https://docs.llamaindex.ai/en/stable/understanding/storing/storing/

# db = chromadb.EphemeralClient() # In Memory
db = chromadb.PersistentClient(path="./chroma_db") # Persistant
chroma_collection = db.get_or_create_collection("DB_collection")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Change default storage context to new vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Creating a vector store index
index = VectorStoreIndex(nodes, # Using nodes (from pipeline) instead of raw documents. Removing .from_documents
                        storage_context=storage_context)


NameError: name 'nodes' is not defined

In [39]:
# Regular Qurey Engine

query_engine = index.as_query_engine()

In [40]:
# Hyde Query Engine

from llama_index.core.indices.query.query_transform.base import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

hyde = HyDEQueryTransform(include_original=True)
query_engine_hyde = TransformQueryEngine(query_engine, query_transform=hyde)

In [41]:
# Multi-Step query Engine

from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine import MultiStepQueryEngine

step_decompose_transform = StepDecomposeQueryTransform(verbose=False)
query_engine_multistep = MultiStepQueryEngine(query_engine, query_transform=step_decompose_transform)

In [45]:
response = query_engine.query("What papers describe the impact of readmission rates. Create a comprehensive list.")
display(Markdown(f"{response}"))

Rewrite

Based on the provided excerpt, several papers describe the impact of readmission rates:

* Krumholz HM, Lin Z, Keenan PS, et al. Relationship between hospital readmission and mortality rates for patients hospitalized with acute myocardial infarction, heart failure, or pneumonia. JAMA.
This paper discusses the relationship between hospital readmissions and mortality rates for patients with acute myocardial infarction, heart failure, or pneumonia.

Other papers that may describe the impact of readmission rates include:

* Bradley EH, Curry L, Horwitz LI, et al. Contemporary evidence about hospital strategies for reducing 30-day readmissions: a national study. J Am Coll Cardiol.
* McCrum ML, Joynt KE, Orav EJ, Gawande AA, Jha AK. Mortality for publicly reported conditions and overall hospital mortality rates. JAMA Intern Med.
* Vidic A, Chibnall JT, Hauptman PJ. Heart failure is a major contributor to hospital readmission penalties. J Card Fail.

These papers may provide additional information on the impact of readmission rates on patient outcomes and healthcare policy.

In [46]:
response = query_engine_hyde.query("What papers describe the impact of readmission rates. Create a comprehensive list.")
display(Markdown(f"{response}"))

**Rewrite**

The original answer provided a comprehensive list of papers that describe the impact of readmission rates. To refine this answer, let's consider the new context: inpatient performance, mortality rates, and readmission rates during pandemics.

In light of this new context, I would like to suggest some papers that specifically address the relationship between workload saturation, hospital capacity, patient flows, pandemics, mortality burden, readmission rates, and inpatient performance. Here are a few examples:

1. **Lau et al. (2019)**: "Excess influenza hospital admissions and costs due to the 2009 H1N1 pandemic in England." This study investigated the impact of the 2009 H1N1 pandemic on hospital admissions, readmission rates, and mortality outcomes.
2. **Madsen et al. (2014)**: "High levels of bed occupancy associated with increased inpatient and thirty-day hospital mortality in Denmark." This study found that high bed occupancy rates were linked to increased mortality rates and readmission rates during pandemics.
3. **Schwierz et al. (2012)**: "Demand, selection, and patient outcomes in German acute care hospitals." This study explored the relationship between demand, selection, and patient outcomes in German acute care hospitals during pandemics.

These papers provide a starting point for understanding the relationships between workload saturation, hospital capacity, patient flows, pandemics, mortality burden, readmission rates, and inpatient performance.

In [47]:
response = query_engine_multistep.query("What papers describe the impact of readmission rates. Create a comprehensive list.")
display(Markdown(f"{response}"))

Yes, there are studies that investigate the relationship between hospital readmission rates and patient outcomes. These studies highlight the significance of socioeconomic status on outcomes, as well as the role of electronic health records (EHRs) in improving patient outcomes and reducing readmissions. The impact of socioeconomic status on readmission rates is particularly important for patients with congestive heart failure, as it can influence treatment adherence, access to care, and overall health outcomes.

One study found that off-service placement was associated with a 13.1% increase in the likelihood of 30-day hospital readmission (EC.2 online). This suggests that patients placed off-service may be at higher risk for readmissions, which can have negative consequences for their health and well-being.

Another study showed that the use of an instrumental variable approach to account for endogenous selection of patients into on- or off-service beds was crucial in understanding the impact of off-service placement on patient outcomes (EC.3 online). This highlights the importance of careful analysis when examining the relationship between off-service placement and readmission rates.

Overall, the literature suggests that off-service placement can have a significant impact on readmission rates, and that careful consideration should be given to this factor in hospital capacity management decisions.

Based on the knowledge source context and previous reasoning, I would suggest that the following papers describe the impact of readmission rates:

* Carey (2015) - study on measuring the hospital length of stay/readmission cost trade-off under a bundled payment mechanism
* Reynolds et al. (2015) - article exploring the relationship between acute heart failure hospital length of stay and subsequent readmission and all-cause mortality
* Rinne et al. (2014) - study examining weekend discharges after COPD hospitalizations and their association with a higher risk of readmission

Please note that this answer is based on the provided context and previous reasoning, which mentions studies on hospital quality initiatives and patient outcomes. If there's more information available or if you'd like me to expand on this list, please let me know!

## Customizing the stages of querying (retriever, node postprocessors, response synthesis)
https://docs.llamaindex.ai/en/stable/understanding/querying/querying/

In [48]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor